<a href="https://colab.research.google.com/github/NHleza/Final_Project/blob/main/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. Data Simulation
class HealthDataSimulator:
    def __init__(self, num_records=100):
        self.num_records = num_records

    def simulate(self):
        data = {
            'timestamp': pd.date_range(start='2023-10-01', periods=self.num_records, freq='T'),
            'heart_rate': np.random.randint(60, 100, self.num_records),
            'blood_oxygen': np.random.randint(90, 100, self.num_records),
            'activity_level': np.random.choice(['low', 'moderate', 'high'], self.num_records)
        }
        return pd.DataFrame(data)

# 2. Anomaly Detection Model
class HealthAnomalyDetector:
    def __init__(self, contamination=0.1):
        self.model = IsolationForest(contamination=contamination, random_state=42)

    def train(self, df):
        features = df[['heart_rate', 'blood_oxygen']]
        self.model.fit(features)
        df['anomaly'] = self.model.predict(features)
        df['anomaly_label'] = df['anomaly'].apply(lambda x: 'Anomaly' if x == -1 else 'Normal')
        return df

    def evaluate(self, df):
        X = df[['heart_rate', 'blood_oxygen']]
        y = df['anomaly'].apply(lambda x: 1 if x == -1 else 0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model.fit(X_train)
        y_pred = self.model.predict(X_test)
        y_pred_labels = np.where(y_pred == -1, 1, 0)
        report = classification_report(y_test, y_pred_labels, target_names=['Normal', 'Anomaly'])
        return report

# 3. Usage
if __name__ == '__main__':
    # Simulate health data
    simulator = HealthDataSimulator()
    df = simulator.simulate()

    # Train anomaly detector
    detector = HealthAnomalyDetector()
    df = detector.train(df)

    # Evaluate the model
    print(detector.evaluate(df))
